In [230]:
import pandas as pd
import os

In [231]:
# Set paths for csv files
csv_path_sc = os.path.join('.', 'Resources', 'schools_complete.csv')
csv_path_st = os.path.join('.', 'Resources', 'students_complete.csv')

In [232]:
# Read in csv files and change column names
schools_df = pd.read_csv(csv_path_sc)
schools_df = schools_df.rename(columns= {'school_name': 'School Name', 'type': 'Type', 'size': 'Total Students', 
                                         'budget': 'Total School Budget'})

students_df = pd.read_csv(csv_path_st)
students_df = students_df.rename(columns= {'student_name': 'Student Name', 'gender': 'Gender', 'grade': 'Grade', 
                                           'school_name': 'School Name', 'reading_score': 'Reading Score', 'math_score': 'Math Score'})

In [233]:
# Merge school and student dataframes
school_data_full = pd.merge(schools_df, students_df, how= 'left', on=['School Name', 'School Name'])
school_data_full.head()

,School ID,School Name,Type,Total Students,Total School Budget,Student ID,Student Name,Gender,Grade,Reading Score,Math Score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [234]:
# District Summary Analysis
# Calculate total number of schools
dist_school_count = len(school_data_full['School Name'].unique())

# Calculate total number of students (sum of student name value count)
dist_student_count = sum(school_data_full['Student Name'].value_counts())

# Calculate total budget (same as above)
dist_total_budget = sum(school_data_full['Total School Budget'].unique())

# Calculate avg math and reading (sum of math/student count etc.)
dist_avg_math = school_data_full['Math Score'].sum() / dist_student_count

dist_avg_reading = school_data_full['Reading Score'].sum() / dist_student_count

# Calculate % passing math with score of 70 or higher
# Find all scores >=70 using loc, find length or that and divide by student count, muptiply by 100
passing_math = (len(school_data_full.loc[school_data_full['Math Score'] >= 70]) / dist_student_count) * 100
passing_math = round(passing_math, 6)


# Calulate % passing reading with score of 70 or higher (see above for logic)
passing_reading = len(school_data_full.loc[school_data_full['Reading Score'] >= 70])/ dist_student_count * 100
passing_reading = round(passing_reading, 6)


# Calculate Overall passing rate - (passing_math + passing_reading)/2
overall_passing = (passing_math + passing_reading) / 2
overall_passing = round(overall_passing, 6)


In [235]:
# Create Summary Table for District Info
dist_summ_df = pd.DataFrame({'Total Schools': [dist_school_count], 'Total Students': dist_student_count, 'Total Budget': dist_total_budget,
                           'Average Math Score': dist_avg_math, 'Average Reading Score': dist_avg_reading, '% Passing Math': passing_math,
                            '% Passing Reading': passing_reading, 'Overall Passing Rate': overall_passing})
dist_summ_df['Total Budget'] = dist_summ_df['Total Budget'].map('${:,.2f}'.format)
dist_summ_df['% Passing Math'] = dist_summ_df['% Passing Math'].map('{:}%'.format)
dist_summ_df['% Passing Reading'] = dist_summ_df['% Passing Reading'].map('{:}%'.format)
dist_summ_df['Overall Passing Rate'] = dist_summ_df['Overall Passing Rate'].map('{:}%'.format)
dist_summ_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853%,85.805463%,80.393158%


In [236]:
# School Summary Analysis
# Group main dataframe by school name
school_groups = school_data_full.groupby(['School Name'])

In [237]:
# Make a new dataframe getting averages of size, budget, and scores
avg_schools = school_groups.mean()
avg_schools = avg_schools[['Total Students', 'Total School Budget', 'Reading Score', 'Math Score']]

In [238]:
# Add column for Per Student Budget (total budget/ total students)
avg_schools['Per Student Budget'] = avg_schools['Total School Budget'] / avg_schools['Total Students']

In [239]:
# Add column of type ************* find a better way for this
avg_schools['Type'] = school_groups['Type'].unique()
# Make the type not show up as []
avg_schools['Type'] = avg_schools['Type'].str[0]

In [240]:
# Create new dataframe with sums of columns from groupby
sum_schools = school_groups.sum()

In [241]:
# Calculate average math score
avg_schools['Average Math Score'] = sum_schools['Math Score'] / avg_schools['Total Students']
avg_schools['Average Reading Score'] = sum_schools['Reading Score'] / avg_schools['Total Students']

In [242]:
# Use merged dataframe to find students passing math and reading (scores over 70)
passing_math_df = school_data_full[school_data_full['Math Score']>= 70]
passing_read_df = school_data_full[school_data_full['Reading Score']>= 70]
# Group new dataframes by school name
group_pass_math = passing_math_df.groupby('School Name')
group_pass_read = passing_read_df.groupby('School Name')


In [243]:
# Add columns to avg_schools dataframe for % passing math and reading (count of math scores in group_pass_math/ total students)
avg_schools['% Passing Math'] = group_pass_math['Math Score'].count()/ avg_schools['Total Students'] * 100
avg_schools['% Passing Reading'] = group_pass_read['Reading Score'].count()/ avg_schools['Total Students'] * 100

In [244]:
# Add column for overall passing (%PassMath + % PassRead)/ 2
avg_schools['% Overall Passing Rate'] = (avg_schools['% Passing Math'] + avg_schools['% Passing Reading']) / 2


In [245]:
# Reorganize column names
org_avg_schools = avg_schools[['Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 
              'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]


In [246]:
# Sort values on % Overall to find top 5 and bottom 5
overall_sort_top = org_avg_schools.sort_values('% Overall Passing Rate', ascending = False)
overall_sort_top.head(5)

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635.0,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [247]:
overall_sort_bottom = org_avg_schools.sort_values('% Overall Passing Rate')
overall_sort_bottom.head()

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999.0,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [248]:
# Math scores by grade
# Filter by grade
grade9_df = school_data_full.loc[school_data_full['Grade'] == '9th', ['School Name', 'Math Score']]
grade10_df = school_data_full.loc[school_data_full['Grade'] == '10th', ['School Name', 'Math Score']]
grade11_df = school_data_full.loc[school_data_full['Grade'] == '11th', ['School Name', 'Math Score']]
grade12_df = school_data_full.loc[school_data_full['Grade'] == '12th', ['School Name', 'Math Score']]


In [249]:
# Group each grade by school and find average
group_grade_9 = grade9_df.groupby('School Name')
avg_grade9 = group_grade_9.mean()
group_grade_10 = grade10_df.groupby('School Name')
avg_grade10 = group_grade_10.mean()
group_grade_11 = grade11_df.groupby('School Name')
avg_grade11 = group_grade_11.mean()
group_grade_12 = grade12_df.groupby('School Name')
avg_grade12 = group_grade_12.mean()

In [250]:
# Create new dataframe including each grade
math_by_grade_1 = pd.merge(avg_grade9, avg_grade10, on='School Name', how= 'left')
math_by_grade_1
math_by_grade_2 = pd.merge(math_by_grade_1, avg_grade11, on='School Name', how = 'left')
# Rename columns now because they will end up duplicating later and I won't be able to rename
math_by_grade_2r = math_by_grade_2.rename(columns= {'Math Score_x': '9th', 'Math Score_y': '10th', 'Math Score': '11th'})

math_by_grade_final = pd.merge(math_by_grade_2r, avg_grade12, on='School Name', how = 'left')

# Rename columns
math_by_grade_finalr = math_by_grade_final.rename(columns= {'Math Score': '12th'})
math_by_grade_finalr

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [251]:
# Reading scores by grade
# Filter by grade
grade9r_df = school_data_full.loc[school_data_full['Grade'] == '9th', ['School Name', 'Reading Score']]
grade10r_df = school_data_full.loc[school_data_full['Grade'] == '10th', ['School Name', 'Reading Score']]
grade11r_df = school_data_full.loc[school_data_full['Grade'] == '11th', ['School Name', 'Reading Score']]
grade12r_df = school_data_full.loc[school_data_full['Grade'] == '12th', ['School Name', 'Reading Score']]


In [252]:
# Group each grade by school and find average
group_grade_9r = grade9r_df.groupby('School Name')
avg_grade9r = group_grade_9r.mean()
group_grade_10r = grade10r_df.groupby('School Name')
avg_grade10r = group_grade_10r.mean()
group_grade_11r = grade11r_df.groupby('School Name')
avg_grade11r = group_grade_11r.mean()
group_grade_12r = grade12r_df.groupby('School Name')
avg_grade12r = group_grade_12r.mean()


In [253]:
# Create new dataframe including each grade
reading_by_grade_1 = pd.merge(avg_grade9r, avg_grade10r, on='School Name', how= 'left')
reading_by_grade_1
reading_by_grade_2 = pd.merge(reading_by_grade_1, avg_grade11r, on='School Name', how = 'left')

# Rename columns now because they will end up duplicating later and I won't be able to rename
reading_by_grade_2r = reading_by_grade_2.rename(columns= {'Reading Score_x': '9th', 'Reading Score_y': '10th', 'Reading Score': '11th'})

# Merge final series to dataframe
reading_by_grade_final = pd.merge(reading_by_grade_2r, avg_grade12r, on='School Name', how = 'left')

# Rename columns
reading_by_grade_finalr = reading_by_grade_final.rename(columns= {'Reading Score': '12th'})
reading_by_grade_finalr

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [254]:
# Scores by School Spending
# Find range of budgets per student usind describe
avg_schools.describe()

,Total Students,Total School Budget,Reading Score,Math Score,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
count,15.000000,1.500000e+01,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,2611.333333,1.643295e+06,82.529188,80.432530,620.066667,80.432530,82.529188,80.987054,89.219023,85.103039
std,1420.915282,9.347763e+05,1.519329,3.374548,28.544368,3.374548,1.519329,13.998824,8.180664,11.071648
min,427.000000,2.480870e+05,80.744686,76.629414,578.000000,76.629414,80.744686,65.683922,79.299014,73.293323
25%,1698.000000,1.046265e+06,81.000178,77.060448,591.500000,77.060448,81.000178,66.523328,81.042716,73.722150
50%,2283.000000,1.319574e+06,83.725724,83.061895,628.000000,83.061895,83.725724,92.505855,95.854628,94.379391
75%,3474.000000,2.228999e+06,83.901965,83.388902,641.500000,83.388902,83.901965,93.629746,96.575376,95.234673
max,4976.000000,3.124928e+06,84.044699,83.839917,655.000000,83.839917,84.044699,94.594595,97.308869,95.586652


In [255]:
# Create bins to cover the range of spending per student (min 578 max 655)

spend_bins = [0, 585, 600, 615, 630, 645, 660]
spend_names = ['<$585', '$585-600', '$600-615', '$615-630', '$630-645', '>$645']
avg_schools['Per Student Spending Ranges'] = pd.cut(avg_schools['Per Student Budget'], spend_bins, labels = spend_names)

#Redo columns to match final product
avg_schools_new = avg_schools[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                               '% Overall Passing Rate', 'Per Student Spending Ranges']]

# Group by  ranges
per_student_bins = avg_schools_new.groupby('Per Student Spending Ranges')
per_student_bins.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Spending Ranges,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-600,83.359455,83.725724,93.867121,95.854628,94.860875
$600-615,83.839917,84.044699,94.594595,95.945946,95.270270
$615-630,80.199966,82.425360,80.036217,89.536122,84.786170
$630-645,78.518855,81.624473,73.484209,84.391793,78.938001
>$645,76.997210,81.027843,66.164813,81.133951,73.649382


In [256]:
# Scores by School Size
# Set bins to cover range of school size 427-4967
size_bins = [0, 1000, 2500, 5000]
size_names = ['Small(<1000)', 'Medium(1000-2500)', 'Large(2500-5000)']
avg_schools['School Size'] = pd.cut(avg_schools['Total Students'], size_bins, labels = size_names)

#Redo columns to match final product
avg_schools_size = avg_schools[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                               '% Overall Passing Rate', 'School Size']]

# Group by  ranges
school_size_bins = avg_schools_size.groupby('School Size')
school_size_bins.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium(1000-2500),83.357937,83.885280,93.644365,96.748840,95.196603
Large(2500-5000),76.956733,80.966636,66.548453,80.799062,73.673757


In [257]:
# Scores by School Type
# Group by school type and run mean
avg_schools_type = avg_schools[['Type', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                               '% Overall Passing Rate', 'School Size']]
schools_type = avg_schools_type.groupby('Type')
schools_type.mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
